In [32]:
from langchain_function_DB import tools,available_functions
from start_model import model_starter
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage,FunctionMessage
from langchain.memory import ConversationBufferMemory
import datetime
from writelog import write_info

In [33]:

llm = model_starter()

In [46]:
# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory = list()
system_prompt = """
    [Today]
    {date}
    You are an agent who is capable of applying multiple [tools] to get to correct information when [previous output] is not enough to answer the question

    you should follow instrucitons below : 

    following are instruciton of this process : 
    1. check time and company name and company number are identified correctly or no , especailly if time is mentioned to be latest then you should match year and quarter between [Today] and [previous output] as priority
    2. confirm  whether [previous output] could answer the query. If it is enough to answer the question then stop the search , otherwise initiate a search to correct the data.
    3. [previous output] contains tools used and its output, be careful with same output from same function.
    4. if the information you try to retrieve is not existing, you will change the quarter and year to fetch another information
    5. before you use API searches information, explain to the user how you will fulfill their requests with the functions provided
    6. please respond in traditional chinese !!!""".format(date = datetime.date.today().strftime("%Y-%m-%d"))
# memory.chat_memory.add_message(SystemMessage(system_prompt)) # system prompt
memory.append(SystemMessage(system_prompt)) # system prompt

llm_with_tools = llm.bind_tools(tools)
# Define a function to manage memory and pass it as context

def run_agent_with_memory(agent, query, memory):
    # memory.chat_memory.add_user_message(HumanMessage(query))

    # covert to langchain class

    # ai_msg = agent.invoke(memory.chat_memory.messages)
    ai_msg = agent.invoke(memory)


    # memory.chat_memory.add_ai_message(ai_msg)
    memory.append(ai_msg)

    return ai_msg

# execute function
def execute_function(agent,ai_msg,query,memory):
    info = list()
    # memory.chat_memory.add_message(AIMessage(ai_msg.additional_kwargs)) # tool_calls first
    while(ai_msg.response_metadata['finish_reason'] == 'tool_calls'):
        for tool_call in ai_msg.tool_calls:
            tool = available_functions[tool_call["name"]]
            tool_msg = tool.invoke(tool_call)
            # memory.chat_memory.add_message(tool_msg)
            memory.append(tool_msg)
            info.append(tool_msg)
        ai_msg = run_agent_with_memory(agent, query, memory)
    return ai_msg
# Generate a response
def response_llm(llm,info,query,memory):
    system_prompt = "請用拿到的資訊針對問題給出回覆，並且以繁體中文給使用者進行回覆: {out}"
    response = llm.invoke([SystemMessage(content=system_prompt.format(out = info)),
                            HumanMessage(content=query)])
    # memory.chat_memory.add_message(response)
    memory.append(response)

    return response


In [49]:
query = '我要精誠的最新財報'
# query = "我想要精誠，台積電跟鴻海的的最新財報"

memory.append(HumanMessage(query))
ai_msg = run_agent_with_memory(llm_with_tools,query, memory)
info = execute_function(llm_with_tools,ai_msg,query,memory)
# response = response_llm(llm,info,query,memory)
for i in info.content.split('\n'):
    print(i)

精誠（公司代號：6214）在2023年第3季的財報如下：

- 營業收入：24,859,631 元
- 營業成本：19,131,844 元
- 營業毛利：5,727,787 元
- 營業費用：4,591,414 元
- 營業利益：1,136,373 元
- 燘外收入及支出：449,672 元
- 稅前淨利：1,586,045 元
- 所得稅費用：302,425 元
- 繼續營業單位本期淨利：1,283,620 元
- 本期淨利：1,283,620 元
- 淨利（歸屬於母公司業主）：1,218,745 元
- 每股盈餘：4.91 元
- 其他綜合損益：146,569 元

如需進一步資訊，請告訴我！


In [50]:
memory

[SystemMessage(content='\n    [Today]\n    2024-11-21\n    You are an agent who is capable of applying multiple [tools] to get to correct information when [previous output] is not enough to answer the question\n\n    you should follow instrucitons below : \n\n    following are instruciton of this process : \n    1. check time and company name and company number are identified correctly or no , especailly if time is mentioned to be latest then you should match year and quarter between [Today] and [previous output] as priority\n    2. confirm  whether [previous output] could answer the query. If it is enough to answer the question then stop the search , otherwise initiate a search to correct the data.\n    3. [previous output] contains tools used and its output, be careful with same output from same function.\n    4. if the information you try to retrieve is not existing, you will change the quarter and year to fetch another information\n    5. before you use API searches information, ex

In [43]:
from financial_func import financial_statement
financial_statement(2024,3,6214)

/Users/fernandodoro/Desktop/修成/financial_func.py:92: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(r.text, header=None)


{'公司 代號': 6214,
 '公司名稱': '精誠',
 '營業費用': 4895774.0,
 '所得稅費用（利益）': '307112',
 '停業單位損益': '--',
 '合併前非屬共同控制股權損益': '--',
 '合併前非屬共同控制股權綜合損益淨額': '--',
 '綜合損益總額歸屬於母公司業主': '1801841',
 '綜合損益總額歸屬於共同控制下前手權益': '--',
 '綜合損益總額歸屬於非控制權益': '67414',
 '基本每股盈餘（元）': 6.29,
 '稅前淨利（淨損）': 1949188.0,
 '繼續營業單位本期淨利（淨損）': 1642076.0,
 '本期淨利（淨損）': 1642076.0,
 '本期綜合損益總額': 1869255.0,
 '淨利（淨損）歸屬於共同控制下前手權益': '--',
 '營業收入': 28298141.0,
 '營業成本': '22286313',
 '原始認列生物資產及農產品之利益（損失）': '--',
 '生物資產當期公允價值減出售成本之變動利益（損失）': '--',
 '營業毛利（毛損）': 6011828.0,
 '未實現銷貨（損）益': '--',
 '已實現銷貨（損）益': '--',
 '營業毛利（毛損）淨額': 6011828.0,
 '其他收益及費損淨額': '--',
 '營業利益（損失）': 1116054.0,
 '營業外收入及支出': 833134.0,
 '其他綜合損益（淨額）': '227179',
 '淨利（淨損）歸屬於母公司業主': '1575070',
 '淨利（淨損）歸屬於非控制權益': '67006',
 'year': 2024,
 'quarter': 3}

In [ ]:
ai_msg

AIMessage(content='為了滿足您的需求，我將進行以下步驟來獲取精誠、台積電和鴻海的最新財報：\n\n1. 首先，我會確認這三間公司的最新財報資料，包括公司名稱、公司編號，以及最新的年份和季度是否正確。\n2. 如果有需要，我會尋找這三間公司的財報資料，並確保它們是最新的。\n\n讓我來查找這三間公司的財報資料。請稍候。', additional_kwargs={'tool_calls': [{'id': 'call_alqMOfUyAJdLP3ag9yttij5T', 'function': {'arguments': '{"query": "精誠"}', 'name': 'find_company_id_data'}, 'type': 'function'}, {'id': 'call_OI3wqzSajtCPOCuN5NLQ6GhP', 'function': {'arguments': '{"query": "台積電"}', 'name': 'find_company_id_data'}, 'type': 'function'}, {'id': 'call_zBpOGN08uHuY7b0uNOgp52iX', 'function': {'arguments': '{"query": "鴻海"}', 'name': 'find_company_id_data'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 182, 'prompt_tokens': 405, 'total_tokens': 587, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-9dd6e633-46cf-412c-9c73-cc667b1

In [ ]:
memory.chat_memory.messages

AttributeError: 'list' object has no attribute 'chat_memory'

In [74]:
memory = list()
memory.append(SystemMessage(system_prompt)) # system prompt


In [80]:
# 如果你指定tool_choice 為某api, finish_reason必定為stop
def execute_function(agent,ai_msg,query,memory):
    info = list()
    # memory.chat_memory.add_message(AIMessage(ai_msg.additional_kwargs)) # tool_calls first
    for tool_call in ai_msg.tool_calls:
        tool = available_functions[tool_call["name"]]
        tool_msg = tool.invoke(tool_call)
        # memory.chat_memory.add_message(tool_msg)
        memory.append(tool_msg)
        info.append(tool_msg)

    return info

In [81]:
query = '我要精誠的最新財報'
# query = "我想要精誠，台積電跟鴻海的的最新財報"
memory.append(HumanMessage(query))

llm_with_tools = llm.bind_tools(tools,tool_choice="find_company_id_data")
ai_msg = run_agent_with_memory(llm_with_tools,query, memory)
info = execute_function(llm_with_tools,ai_msg,query,memory)

print('mid')

llm_with_tools = llm.bind_tools(tools,tool_choice="financial_property_data")
ai_msg = run_agent_with_memory(llm_with_tools,query, memory)
info = execute_function(llm_with_tools,ai_msg,query,memory)

ai_msg = response_llm(llm,info,query,memory)
for i in ai_msg.content.split('\n'):
    print(i)

2024-11-21 13:52:33.429 | DEBUG    | chat_openai:chat_with_aoai_gpt:224 - get_resouce_time=1.0203180313110352 ChatCompletionTime=5.732229948043823
2024-11-21 13:52:33,433 - INFO - prompt_tokens: 58153
2024-11-21 13:52:33,436 - INFO - complete_tokens: 18
2024-11-21 13:52:33,623 - INFO - company info
2024-11-21 13:52:33,624 - INFO - 精誠
2024-11-21 13:52:33,624 - INFO - 6214


mid


/Users/fernandodoro/Desktop/修成/financial_func.py:92: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(r.text, header=None)


根據最新的財報資訊，精誠（公司代號：6214）的財務狀況如下：

- 營業收入：28,298,141 元
- 營業成本：22,286,313 元
- 營業毛利：6,011,828 元
- 營業費用：4,895,774 元
- 營業利益：1,116,054 元
- 營業外收入及支出：833,134 元
- 稅前淨利：1,949,188 元
- 所得稅費用：307,112 元
- 本期淨利：1,642,076 元
- 淨利（歸屬於母公司業主）：1,575,070 元
- 基本每股盈餘：6.29 元
- 總綜合損益總額：1,869,255 元

如需更詳細的資訊或進一步的分析，請告訴我！


In [82]:
memory

[SystemMessage(content='\n    [Today]\n    2024-11-21\n    You are an agent who is capable of applying multiple [tools] to get to correct information when [previous output] is not enough to answer the question\n\n    you should follow instrucitons below : \n\n    following are instruciton of this process : \n    1. check time and company name and company number are identified correctly or no , especailly if time is mentioned to be latest then you should match year and quarter between [Today] and [previous output] as priority\n    2. confirm  whether [previous output] could answer the query. If it is enough to answer the question then stop the search , otherwise initiate a search to correct the data.\n    3. [previous output] contains tools used and its output, be careful with same output from same function.\n    4. if the information you try to retrieve is not existing, you will change the quarter and year to fetch another information\n    5. before you use API searches information, ex

In [83]:
ai_msg

AIMessage(content='根據最新的財報資訊，精誠（公司代號：6214）的財務狀況如下：\n\n- 營業收入：28,298,141 元\n- 營業成本：22,286,313 元\n- 營業毛利：6,011,828 元\n- 營業費用：4,895,774 元\n- 營業利益：1,116,054 元\n- 營業外收入及支出：833,134 元\n- 稅前淨利：1,949,188 元\n- 所得稅費用：307,112 元\n- 本期淨利：1,642,076 元\n- 淨利（歸屬於母公司業主）：1,575,070 元\n- 基本每股盈餘：6.29 元\n- 總綜合損益總額：1,869,255 元\n\n如需更詳細的資訊或進一步的分析，請告訴我！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 230, 'prompt_tokens': 642, 'total_tokens': 872, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-48fa95f8-bea3-4d74-b6ff-bef7e2ebdff8-0', usage_metadata={'input_tokens': 642, 'output_tokens': 230, 'total_tokens': 872, 'input_token_details': {}, 'output_token_details': {}})